# Grid Search hyper parameters for SRST

1. Import dependencies

In [ ]:
import random
import torch
import numpy as np
import pandas as pd
import multiprocessing
import time

from torch import nn
from typing import Sequence

from src.model.grid_search import GridSearchAdapter
from src.util.torch_device import resolve_torch_device
from src.data.indian_pines import load_indian_pines
from src.pipeline.spatial_regulated_self_training_pipeline import (
    SpatialRegulatedSelfTrainingPipeline,
    SpatialRegulatedSelfTrainingPipelineArgs,
    KMeansClustering,
    ClassificationFeatureExtractor,
)
from src.trainer.classification_trainer import ClassificationTrainer
from src.model.fully_convolutional_lenet import FullyConvolutionalLeNet
from src.util.semi_guided import sample_from_segmentation_matrix
from src.util.over_clustering import exponential_decay_over_clustering
from src.definitions import GREED_SEARCH_FOLDER
from src.model.grid_search import GridSearch

2. Prepare env

In [ ]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

generator = torch.Generator()
generator.manual_seed(random_seed)

In [ ]:
f"Device is {device}"

# Indian pines (Cluster exponential decay)

0. Set params

In [ ]:
examples_per_class = 15
epoch_seconds = int(time.time())
run_name = f"indian-pines-cluster-exponential-decay-{epoch_seconds}"

In [ ]:
cpu_count = multiprocessing.cpu_count()

f"Setting num_workers to {cpu_count}"

1. Load dataset

In [ ]:
image, labels = load_indian_pines()

In [ ]:
num_classes = len(np.unique(labels))

f"Number of classes {num_classes}"

In [ ]:
masked_labels = sample_from_segmentation_matrix(labels, examples_per_class)

2. Train model

In [ ]:
class SpatialRegulatedSelfTrainingPipelineGridSearchAdapter(
    GridSearchAdapter[SpatialRegulatedSelfTrainingPipeline]
):

    def params_grid(self) -> dict[str, Sequence[float]]:
        return {
            "splits": [4],
            "learning_rate": [1e-3, 1e-4, 1e-5],
            "patch_size": [9],
            "num_epochs": [11],
            "feature_extractor_epochs": [1, 5, 9, 11],
            "semantic_threshold": [0.5, 0.6, 0.7, 0.8],
            "lambda_v": [0.07, 0.09, 0.2, 0.3, 0.4, 0.49],
            "k_star": [num_classes * 2, num_classes * 3],
            "batch_size": [64]
        }

    def init_model(self, params: dict[str, float]):
        _, _, c = image.shape

        input_channels = int(c / params["splits"])

        model = FullyConvolutionalLeNet(input_channels, num_classes).to(device)

        trainer = ClassificationTrainer(
            num_epochs=params["feature_extractor_epochs"],
            learning_rate=params["learning_rate"],
            loss_fun=nn.CrossEntropyLoss(),
        )

        k_values = exponential_decay_over_clustering(
            k_star=params["k_star"],
            lambda_v=params["lambda_v"],
            max_iter=params["num_epochs"],
        )

        args = SpatialRegulatedSelfTrainingPipelineArgs(
            num_classes=num_classes,
            cluster_sizes=k_values,
            feature_extractor=ClassificationFeatureExtractor(
                model, trainer, generator, batch_size=params["batch_size"]
            ),
            clustering=KMeansClustering(seed=random_seed),
            splits=params["splits"],
            patch_size=params["patch_size"],
            init_patch_size=5,
            semantic_threshold=params["semantic_threshold"],
            spatial_threshold=8,
            spatial_constraint_weights=[1, 0.5],
            record_step_snapshots=True,
        )

        return SpatialRegulatedSelfTrainingPipeline(args, device)

    def fit_model(self, model: SpatialRegulatedSelfTrainingPipeline):
        model.fit(image, masked_labels, labels)

    def score_model(
        self, model: SpatialRegulatedSelfTrainingPipeline
    ) -> list[dict[str, float]]:
        return [it.metrics.__dict__ for it in model.history]

In [ ]:
log_file = GREED_SEARCH_FOLDER / f"{run_name}.csv"

search = GridSearch(
    adapter=SpatialRegulatedSelfTrainingPipelineGridSearchAdapter(),
    optimize_metric="kappa_score",
    log_file=log_file,
)

In [ ]:
_, best_params, best_score = search.run()

In [ ]:
print("Best Params:", best_params)
print("Best Score:", best_score)

3. Training results

In [ ]:
report = pd.read_csv(log_file)

report.head()